In [ ]:
## first, make sure that your conda is setup properly with the right environment
## for that, check that `which conda`, `which pip` and `which python` points to the
## right path. From a clean conda env, this is what you need to do

# conda create --name fcos python=3.8
# conda activate fcos

In [ ]:
## Update with mamba: https://www.anaconda.com/blog/a-faster-conda-for-a-growing-community
# Conda 22.11 update: The libmamba solver's experimental flag has been removed. To use the new solver, update conda in your base environment:
# conda update -n base conda

## To install and set the new solver, run the following commands:
# conda install -n base conda-libmamba-solver
# conda config --set solver libmamba

In [ ]:
## Install jupyter lab: https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html
# conda install -c conda-forge jupyterlab
# jupyter lab

In [32]:
!conda --version
!python --version
!conda list python

conda 22.11.1
Python 3.8.15
# packages in environment at /home/aru/miniconda3/envs/fcos:
#
# Name                    Version                   Build  Channel
ipython                   8.7.0            py38h06a4308_0  
ipython_genutils          0.2.0                      py_1    conda-forge
opencv-python             4.5.3.56                 pypi_0    pypi
python                    3.8.15               h7a1cb2a_2  
python-dateutil           2.8.2              pyhd8ed1ab_0    conda-forge
python-fastjsonschema     2.16.2             pyhd8ed1ab_0    conda-forge
python_abi                3.8                      2_cp38    conda-forge


In [33]:
!nvidia-smi
!nvcc --version

Wed Jan 11 01:57:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.12    Driver Version: 527.41       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| N/A   41C    P8    13W /  N/A |   1628MiB /  6144MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [35]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)

1.13.1
True
1
0
NVIDIA GeForce RTX 3060 Laptop GPU
11.6


In [4]:
# FCOS and coco api dependencies
!pip install ninja yacs cython matplotlib tqdm

# follow PyTorch installation in https://pytorch.org/get-started/locally/
# we give the instructions for CUDA 10.2
# !conda install pytorch torchvision torchaudio pytorch-cuda=11.6 -c pytorch -c nvidia

# cannot done here since need [y]/n input, done in terminal

In [6]:
!pip install opencv-python==4.5.3.56 scikit-image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 355.7 kB/s eta 0:00:0000:0100:04
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 356.7 kB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 345.4 kB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 359.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 356.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.3/210.3 kB 377.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 353.6 kB/s eta 0:00:0000:0100:02


In [7]:
# install pycocotools. Please make sure you have installed cython.
!git clone https://github.com/cocodataset/cocoapi.git

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 354.00 KiB/s, done.
Resolving deltas: 100% (576/576), done.


In [8]:
%cd cocoapi/PythonAPI
!python setup.py build_ext install

/home/aru/miniconda3/envs/fcos/cocoapi/PythonAPI
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/home/aru/miniconda3/envs/fcos/lib/python3.8/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/aru/miniconda3/envs/fcos/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-cpython-38
creating build/temp.linux-x86_64-cpython-38/pycocotools
gcc -pthread -B /home/aru/miniconda3/envs/fcos/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/home/aru/miniconda3/envs/fcos/lib/python3.8/site-packages/numpy/core/include -I../common -I/home/aru/miniconda3/envs/fcos/include/python3.8 -c ../common/maskApi.c -o build/temp.linux-x86_64-cpython-38/../common/mas

In [9]:
# install PyTorch Detection
%cd ../..
!git clone https://github.com/tianzhi0549/FCOS.git
%cd FCOS

/home/aru/miniconda3/envs/fcos
Cloning into 'FCOS'...
remote: Enumerating objects: 2102, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 2102 (delta 6), reused 11 (delta 5), pack-reused 2090
Receiving objects: 100% (2102/2102), 8.81 MiB | 358.00 KiB/s, done.
Resolving deltas: 100% (1370/1370), done.
/home/aru/miniconda3/envs/fcos/FCOS


In [40]:
!python setup.py build develop --no-deps

running build
running build_py
running egg_info
writing fcos.egg-info/PKG-INFO
writing dependency_links to fcos.egg-info/dependency_links.txt
writing requirements to fcos.egg-info/requires.txt
writing top-level names to fcos.egg-info/top_level.txt
reading manifest file 'fcos.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'fcos.egg-info/SOURCES.txt'
/home/aru/miniconda3/envs/fcos/lib/python3.8/site-packages/setuptools/command/build_py.py:202: SetuptoolsDeprecationWarning:     Installing 'fcos.bin' as data is deprecated, please list it in `packages`.
    !!


    ############################
    # Package would be ignored #
    ############################
    Python recognizes 'fcos.bin' as an importable package,
    but it is not listed in the `packages` configuration of setuptools.

    'fcos.bin' has been automatically added to the distribution only
    because it may contain data files, but this behavior is likely to

In [29]:
!gcc --version
!g++ --version

gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

g++ (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [42]:
!fcos https://github.com/tianzhi0549/FCOS/raw/master/demo/images/COCO_val2014_000000000885.jpg

Downloading: "https://cloudstor.aarnet.edu.au/plus/s/3GKwaxZhDSOlCZ0/download#fcos_syncbn_bs32_c128_MNV2_FPN_1x.pth" to /home/aru/.torch/models/fcos_syncbn_bs32_c128_MNV2_FPN_1x.pth
100%|██████████████████████████████████████| 15.0M/15.0M [00:03<00:00, 4.50MB/s]
/home/aru/miniconda3/envs/fcos/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525552411/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
person           confidence: 0.81    bbox: 519.2 345.5 770.4 747.2
person           confidence: 0.81    bbox: 518.3 344.3 771.1 741.1
person           confidence: 0.80    bbox: 519.6 343.6 769.2 749.1
person           confidence: 0.78    bbox: 519.1 342.5 771.4 743.4
tennis racket    confidence: 0.77    bbox: 737.3 502.8 897.9 579.1
tennis racket    confidenc

In [4]:
%cd ~
%cd miniconda3/envs/fcos/FCOS

/home/aru
/home/aru/miniconda3/envs/fcos/FCOS


In [16]:
!unzip

UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  test compressed archive data
  -u  update files, create if necessary      -z  display archive comment only
  -v  list verbosely/show version info       -T  timestamp archive to latest
  -x  exclude files that follow (in xlist)   -d  extract files into exdir
modifiers:
  -n  never overwrite existing files         -q  quiet mode (-qq => quieter)
  -o  overwrite files WITHOUT prompting      -a  auto-convert any text files
  -j  junk paths (do not make directories)   -aa treat ALL files as text
  -U  use escapes for all non-ASCII Unicode  -UU ignore any Unicode fields
  -C  mat

In [ ]:
# need to download coco dataset: https://github.com/tianzhi0549/FCOS/blob/master/MASKRCNN_README.md#perform-training-on-coco-dataset
# use 2014 version

# https://gist.github.com/mkocabas/a6177fc00315403d31572e17700d7fd9
# this one use 2017, change to 2014

!mkdir datasets
%cd datasets

!mkdir coco
%cd coco
!mkdir images
%cd images

!wget http://images.cocodataset.org/zips/train2014.zip
!wget http://images.cocodataset.org/zips/val2014.zip
!wget http://images.cocodataset.org/zips/test2014.zip
!wget http://images.cocodataset.org/zips/unlabeled2014.zip

!unzip train2017.zip
!unzip val2017.zip
!unzip test2017.zip
!unzip unlabeled2017.zip

# if you want to keep the raw files
# !rm train2017.zip
# !rm val2017.zip
# !rm test2017.zip
# !rm unlabeled2017.zip 

%cd ../..
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!wget http://images.cocodataset.org/annotations/stuff_annotations_trainval2017.zip
!wget http://images.cocodataset.org/annotations/image_info_test2017.zip
!wget http://images.cocodataset.org/annotations/image_info_unlabeled2017.zip

!unzip annotations_trainval2017.zip
!unzip stuff_annotations_trainval2017.zip
!unzip image_info_test2017.zip
!unzip image_info_unlabeled2017.zip

# if you want to keep the raw files
# !rm annotations_trainval2017.zip
# !rm stuff_annotations_trainval2017.zip
# !rm image_info_test2017.zip
# !rm image_info_unlabeled2017.zip

In [ ]:
# link the folder to some path

!ln -s /path_to_coco_dataset/annotations datasets/coco/annotations
!ln -s /path_to_coco_dataset/train2014 datasets/coco/train2014
!ln -s /path_to_coco_dataset/test2014 datasets/coco/test2014
!ln -s /path_to_coco_dataset/val2014 datasets/coco/val2014

In [ ]:
!python tools/test_net.py \
    --config-file configs/fcos/fcos_syncbn_bs32_c128_MNV2_FPN_1x.yaml \
    MODEL.WEIGHT FCOS_syncbn_bs32_c128_MNV2_FPN_1x.pth \
    TEST.IMS_PER_BATCH 1

# final output:
#   FileNotFoundError: [Errno 2] No such file or directory: 'datasets/coco/annotations/instances_minival2014.json'